In [90]:
import pickle
import pandas as pd

In [91]:
df = pd.read_csv("data/heart.csv")
df.head(2)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1


Abrimos todos los *transformers* que guardamos cuando hicimos el modelo

In [92]:
# encoding ChestPainType
with open('data/encondingChestPainType.pkl', 'rb') as chest:
    encoding_chestpain = pickle.load(chest)
    
# encoding Angina
with open('data/encondingExerciseAngina.pkl', 'rb') as angina:
    encoding_angina = pickle.load(angina)

    
# encoding Fasting
with open('data/encondingFastingBS.pkl', 'rb') as fasting:
    encoding_fasting = pickle.load(fasting)

# encoding Resting
with open('data/encondingRestingECG.pkl', 'rb') as resting:
    encoding_resting = pickle.load(resting)

# encoding Slope
with open('data/encondingST_Slope.pkl', 'rb') as slope:
    encoding_slope = pickle.load(slope)

# encoding Sex
with open('data/encondingSex.pkl', 'rb') as sex:
    encoding_sex = pickle.load(sex)

# estandarización
with open('data/estandarizacion.pkl', 'rb') as estandarizacion:
    estandarizacion = pickle.load(estandarizacion)


In [93]:
# cargamos el mejor modelo
with open('data/mejor_modelo.pkl', 'rb') as modelo:
    modelo = pickle.load(modelo)

Nos preparamos para hacer la predicción

In [94]:
# recordamos como eran los datos que nos pasaban de origen
df.sample(4)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
675,57,M,NAP,150,126,1,Normal,173,N,0.2,Up,0
599,56,M,ASY,130,203,1,Normal,98,N,1.5,Flat,1
703,41,F,ATA,126,306,0,Normal,163,N,0.0,Up,0
266,52,M,ASY,130,298,0,Normal,110,Y,1.0,Flat,1


In [95]:
# le pedimos al médico que nos de los datos del paciente
resultados = {"Age" : int(input("¿Cuál es tu edad?")),
"Sex" : input("¿Cuál es tu genero?"),
"ChestPainType" : input("¿Tuviste dolor de pecho?"),
"RestingBP" : int(input("¿Cual es tu presion arterial?")),
"Cholesterol" : int(input("¿Cual es tu colesterol?")),
"FastingBS" : int(input("¿Cual es tu nivel de azucar?")),
"RestingECG" : input("¿Cual es tu resultado del electrocardiograma?"),
"MaxHR" : int(input("¿Cual es tu frecuencia cardiaca?")),
"ExerciseAngina" : input("¿Angina inducida?"),
"Oldpeak" : float(input("oldpeak?")),
"ST_Slope" : input("¿Cómo era la pendiente?")}


In [96]:
# convertimos los datos del paciente a dataframe para poder trabajar con ellos. 
df_resultados = pd.DataFrame(resultados, index=[0])
df_resultados

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,23,M,NAP,234,198,1,Normal,123,Y,1.9,Up


In [98]:
# estandarizamos todas las columnas que son de tipo numérico usando el transformer que nos guardamos cuando hicimos el modelo

estandarizadas = pd.DataFrame(estandarizacion.transform(df_resultados[["Age", "RestingBP", "Cholesterol", 
                           "MaxHR", "Oldpeak"]]), columns = ["Age", "RestingBP", "Cholesterol", 
                           "MaxHR", "Oldpeak"])
estandarizadas

,Age,RestingBP,Cholesterol,MaxHR,Oldpeak
0,-3.236379,6.602245,-0.851614,-0.554562,1.118123


In [99]:
df_resultados.head(1)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,23,M,NAP,234,198,1,Normal,123,Y,1.9,Up


In [100]:
# hacemos el encoding del resto de las variables

chest = pd.DataFrame(encoding_chestpain.transform(df_resultados[["ChestPainType"]]), columns= ["ChestPainType"])
angina = pd.DataFrame(encoding_angina.transform(df_resultados[["ExerciseAngina"]]), columns= ["ExerciseAngina"])
resting = pd.DataFrame(encoding_resting.transform(df_resultados[["RestingECG"]]), columns= ["RestingECG"])
slope = pd.DataFrame(encoding_slope.transform(df_resultados[["ST_Slope"]]), columns= ["ST_Slope"])
fasting = pd.DataFrame(encoding_fasting.transform(df_resultados[["FastingBS"]]), columns= ["FastingBS"])
sex = pd.DataFrame(encoding_sex.transform(df_resultados[["Sex"]]).toarray(), columns=["Sex_F", "Sex_M"])


In [101]:
# unimos todos los conjuntos de datos que hemos ido generando con los transformers
df_final = pd.concat([estandarizadas, sex, chest, angina, resting, slope, fasting], axis = 1)
df_final

,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType,ExerciseAngina,RestingECG,ST_Slope,FastingBS
0,-3.236379,6.602245,-0.851614,-0.554562,1.118123,0.0,1.0,1,1,0,1,1


In [102]:
# cambiamos el orden de las columnas ya que estas deben estar en el mismo orden de como entrenamos al modelo
df_final = df_final.reindex(['Age', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'Sex_F', 'Sex_M'], axis=1)

In [104]:
# hacemos la predicción para nuestro nuevo paciente
modelo.predict(df_final)

array([1])

In [105]:
# ¿Cuál es la probabilidad de que le un infarto al paciente?

modelo.predict_proba(df_final)

array([[0.42040418, 0.57959582]])